In [1]:
import pandas as pd

### Read in population data, rename columns and drop an unused one

In [2]:
df_pop = pd.read_csv("../data/munich_pop.csv").rename(columns={"plz": "zipcode",
                                                              "einwohner": "population",
                                                              "qkm": "sqkm",
                                                              "Density": "population_density"}
                                                     )
df_pop.drop("note", axis=1, inplace=True)
df_pop.astype({'zipcode':'str'})
df_pop.head()

,zipcode,population,sqkm,lat,lon,population_density
0,80331,4741,0.782770,48.13575,11.57351,6056.696092
1,80333,11265,1.615148,48.14495,11.56824,6974.593040
2,80335,9042,1.697719,48.14657,11.55112,5325.969728
3,80336,8103,1.568529,48.13101,11.55228,5165.986730
4,80337,15280,1.046991,48.12675,11.55904,14594.203771


In [3]:
df_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   zipcode             75 non-null     int64  
 1   population          75 non-null     int64  
 2   sqkm                75 non-null     float64
 3   lat                 75 non-null     float64
 4   lon                 75 non-null     float64
 5   population_density  75 non-null     float64
dtypes: float64(4), int64(2)
memory usage: 3.6 KB


In [4]:
df_pop.zipcode.nunique()

75

### Read in physicians data and count physicians per zipcode

In [5]:
df_phys = pd.read_csv("../data/physicians_with_coordinates.csv", index_col=0)
df_phys.head()

,name,expertise,zipcode,area,street_and_house_no,telephone,distance,lat,lon
0,Frau Dr. med. Christa Hutterer,"Fachärztin für Diagnostische Radiologie, Fachä...",80331,München-Altstadt-Lehel,Kaufingerstraße 15,089 / 2 00 01 43 - 50,0.1 km,48.137815,11.571804
1,Herr Dr. med. Josef J. Dohrenbusch,Facharzt für Allgemeinmedizin,80331,München,Kaufingerstraße 12,089 / 24 20 93 98,0.1 km,48.137615,11.573961
2,Frau Dr. med. Heidi Herrmann,Fachärztin für Innere Medizin,80331,München,Altheimer Eck 2,089 / 45 22 81 81,0.1 km,48.137541,11.571022
3,Herr Dr. med. (univ.) Thomas Wendel,Praktischer Arzt,80331,München,Altheimer Eck 10,089 / 89 67 40 20,0.1 km,48.137505,11.570328
4,Frau Dr. med. Sabine Konz,Praktische Ärztin,80331,München,Eisenmannstraße 4,089 / 37 02 97 67,0.2 km,48.137763,11.569028


In [6]:
df_phys_by_zipcode = df_phys.groupby("zipcode").agg(physicians_count=('name','count')).reset_index()
df_phys_by_zipcode.head()

,zipcode,physicians_count
0,80331,174
1,80333,102
2,80335,71
3,80336,108
4,80337,51


In [7]:
df_phys_by_zipcode.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   zipcode           90 non-null     int64
 1   physicians_count  90 non-null     int64
dtypes: int64(2)
memory usage: 1.5 KB


In [8]:
df_phys_by_zipcode.zipcode.nunique()

90

### Merge population data with physician counts by zipcode

Lost only one zipcode from population data - not that bad.

In [9]:
df_merged = df_pop.merge(df_phys_by_zipcode)
# Uncomment the next line to keep all zipcodes from population data, filling up number of physicians with 0 where zipcode is missing
#df_merged = df_pop.merge(df_phys_by_zipcode, how="left").fillna(0)
df_merged.head()

,zipcode,population,sqkm,lat,lon,population_density,physicians_count
0,80331,4741,0.782770,48.13575,11.57351,6056.696092,174
1,80333,11265,1.615148,48.14495,11.56824,6974.593040,102
2,80335,9042,1.697719,48.14657,11.55112,5325.969728,71
3,80336,8103,1.568529,48.13101,11.55228,5165.986730,108
4,80337,15280,1.046991,48.12675,11.55904,14594.203771,51


In [10]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74 entries, 0 to 73
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   zipcode             74 non-null     int64  
 1   population          74 non-null     int64  
 2   sqkm                74 non-null     float64
 3   lat                 74 non-null     float64
 4   lon                 74 non-null     float64
 5   population_density  74 non-null     float64
 6   physicians_count    74 non-null     int64  
dtypes: float64(4), int64(3)
memory usage: 4.2 KB


In [11]:
df_merged.zipcode.nunique()

74

In [12]:
no_physicians = df_merged[df_merged["physicians_count"] == 0.0]
no_physicians

,zipcode,population,sqkm,lat,lon,population_density,physicians_count


In [13]:
no_physicians_zipcodes = list(no_physicians.zipcode.values)
no_physicians_zipcodes

[]

### Add columns with physicians density (number of physicians per sqkm) and number of physicians per 1000 people

In [14]:
df_merged["physicians_density"] = df_merged.physicians_count / df_merged.sqkm
df_merged["physicians_per_1000_people"] = df_merged.physicians_count / df_merged.population * 1000
df_merged.head()

,zipcode,population,sqkm,lat,lon,population_density,physicians_count,physicians_density,physicians_per_1000_people
0,80331,4741,0.782770,48.13575,11.57351,6056.696092,174,222.287517,36.701118
1,80333,11265,1.615148,48.14495,11.56824,6974.593040,102,63.152107,9.054594
2,80335,9042,1.697719,48.14657,11.55112,5325.969728,71,41.820820,7.852245
3,80336,8103,1.568529,48.13101,11.55228,5165.986730,108,68.854321,13.328397
4,80337,15280,1.046991,48.12675,11.55904,14594.203771,51,48.711020,3.337696


In [15]:
df_merged.to_csv("../data/merged_dataset.csv")

In [16]:
len(df_merged)

74